In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load Model

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'], trust_remote_code=True)
tokenizer.padding_side = "right"  # default untuk GPT-style model

base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])


# Path model hasil fine-tuning
output_dir = "/content/third_outputs/checkpoint-900"
# Load LoRA weights ke model dasar
model_load = PeftModel.from_pretrained(base_model, output_dir)
model_load.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              

In [ ]:
text = "instruction : Given the math question, student answer, and their explanation, determine if there is a misconception and classify it.\nValid categories: True_Correct, True_Neither, True_Misconception, False_Neither, False_Misconception, False_Correct\nValid misconceptions (only when applicable): NA, Incomplete, WNB, SwapDividend, Mult, FlipChange, Irrelevant, Wrong_Fraction, Additive, Not_variable, Adding_terms, Inverse_operation, Inversion, Duplication, Wrong_Operation, Whole_numbers_larger, Longer_is_bigger, Ignores_zeroes, Shorter_is_bigger, Wrong_fraction, Adding_across, Denominator-only_change, Incorrect_equivalent_fraction_addition, Division, Subtraction, Unknowable, Definition, Interior, Positive, Tacking, Wrong_term, Firstterm, Base_rate, Multiplying_by_4, Certainty, Scale\nFormat your answer as: Category[:Misconception], input: Question: Dots have been arranged in these patterns: [Image: Pattern 1 consists of 6 dots, Pattern 2 consists of 10 dots, Pattern 3 consists of 14 dots and Pattern 4 consists of 18 dots] How many dots would there be in Pattern 6 ?\nAnswer: 26\nExplanation: going up by 4 each set.\n"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model_load.generate(**inputs, max_new_tokens=20)
decoder = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoder)

instruction : Given the math question, student answer, and their explanation, determine if there is a misconception and classify it.
Valid categories: True_Correct, True_Neither, True_Misconception, False_Neither, False_Misconception, False_Correct
Valid misconceptions (only when applicable): NA, Incomplete, WNB, SwapDividend, Mult, FlipChange, Irrelevant, Wrong_Fraction, Additive, Not_variable, Adding_terms, Inverse_operation, Inversion, Duplication, Wrong_Operation, Whole_numbers_larger, Longer_is_bigger, Ignores_zeroes, Shorter_is_bigger, Wrong_fraction, Adding_across, Denominator-only_change, Incorrect_equivalent_fraction_addition, Division, Subtraction, Unknowable, Definition, Interior, Positive, Tacking, Wrong_term, Firstterm, Base_rate, Multiplying_by_4, Certainty, Scale
Format your answer as: Category[:Misconception], input: Question: Dots have been arranged in these patterns: [Image: Pattern 1 consists of 6 dots, Pattern 2 consists of 10 dots, Pattern 3 consists of 14 dots and